# Converting from LSST Skymaps

## Why do we want to convert skymaps in the first place?

If we want to use the [lsst.skymap](https://github.com/lsst/skymap) type skymap that LSST uses, we may only use it in environments that have access to the LSST stack. 

And unfortunately, downloading the stack is non-trivial.

We *could* try to shim some of the LSST-stack dependencies that lsst.skymaps has, but skymaps use a lot of spherical geometry that would require essentially rewriting a number of other packages from scratch.

```bash
lsst.skymap  
├── Required Core Dependencies  
│   ├── lsst.sphgeom ................ Spherical geometry library  
│   ├── lsst.geom ................... Basic geometry primitives  
│   └── lsst.afw .................... Application Framework (WCS)  
│
├── Required Support Dependencies  
│   ├── lsst.pex.config ............. Configuration system  
│   └── lsst.pex.exceptions ......... Exception handling  
│
└── Optional Dependencies  
    └── lsst.daf.butler ............. Data Access Framework  
                                      (used in packers.py only)  
```

*See the Skymaps Overview notebook for a little more explanation about why skymaps need these kinds of math-heavy packages.*

## What do we do to convert the skymap?

### Download the LSST skymap

This next section assumes you have the required access to LSST's The Butler.  

If not, you don't actually really need to be reading this notebook--we've included an already-converted LSST skymap in this package.

*(See the "Reading and using a converted skymap" notebook for an explanation on using the LSST skymap preset).*

In [1]:
import lsst.daf.butler as daf_butler

In [2]:
# Query the Butler for skymaps.
config = "/repo/embargo"
butler = daf_butler.Butler(config, collections="LSSTCam/runs/DRP/20250415_20250422/d_2025_04_23/DM-50409")
refs = butler.registry.queryDatasets("skyMap")

# We can see all the skymaps available in this Butler repository.
list(refs)

[DatasetRef(DatasetType('skyMap', {skymap}, SkyMap), {skymap: 'DC2_cells_v1'}, run='skymaps', id=b7d4b4f1-e108-4570-bbf5-da6093179ac8),
 DatasetRef(DatasetType('skyMap', {skymap}, SkyMap), {skymap: 'hsc_rings_v1'}, run='skymaps', id=6e7fc928-0dc3-4e1b-ae63-5fb63be46216),
 DatasetRef(DatasetType('skyMap', {skymap}, SkyMap), {skymap: 'latiss_v1'}, run='skymaps', id=f677ff86-2b1c-4094-9dad-e39395052cd1),
 DatasetRef(DatasetType('skyMap', {skymap}, SkyMap), {skymap: 'lsst_cells_v1'}, run='skymaps', id=857e0e80-8d08-44f4-a81b-84227b064d5b),
 DatasetRef(DatasetType('skyMap', {skymap}, SkyMap), {skymap: 'ops_rehersal_prep_2k_v1'}, run='skymaps', id=8f5f7e7a-c647-4a88-b8e7-6e2406d071b8),
 DatasetRef(DatasetType('skyMap', {skymap}, SkyMap), {skymap: 'discrete_comcam_rgb_ecdfs_v1'}, run='skymaps', id=1b399fae-8a7e-4633-833d-b713a93d44a3),
 DatasetRef(DatasetType('skyMap', {skymap}, SkyMap), {skymap: 'discrete_comcam_rgb_seagull_v1'}, run='skymaps', id=ca37ba9c-f4ff-4ab1-a27b-7930f4f7739b),
 Data

In [3]:
skymap_name = "lsst_cells_v1"
skymap = next((ref for ref in refs if ref.dataId["skymap"] == skymap_name), None)
if skymap is None:
    raise ValueError(f"No skymap found with the name '{skymap_name}'.")
print(f"Found skymap: {skymap.dataId['skymap']} with ID {skymap.id}")

butler.get(skymap)
skymap_uri = butler.getURI(skymap)
print(f"Skymap URI: {skymap_uri}")

Found skymap: lsst_cells_v1 with ID 857e0e80-8d08-44f4-a81b-84227b064d5b
Skymap URI: s3://embargo@rubin-summit-users/skymaps/skyMap/skyMap_lsst_cells_v1_skymaps.pickle


In [4]:
import os
from pathlib import Path

from lsst.resources import ResourcePath

home_dir = Path.home()
target_dir = home_dir / "skymap-convert" / "tests" / "data" / "raw_skymaps"
base = ResourcePath(target_dir, forceDirectory=True)
new_uri = base.join(skymap_uri.basename())
if os.path.exists(new_uri.path):
    print(f"File {new_uri} already exists. Skipping download.")
else:
    new_uri.transfer_from(skymap_uri, transfer="copy")
    print(f"File {new_uri} downloaded successfully.")

File file:///sdf/home/o/olynn/skymap-convert/tests/data/raw_skymaps/skyMap_lsst_cells_v1_skymaps.pickle already exists. Skipping download.


### Reading the raw skymap

In [5]:
from pathlib import Path

from skymap_convert.utils import load_pickle_skymap

package_root = Path.home() / "skymap-convert"

raw_skymaps_dir = package_root / "tests" / "data" / "raw_skymaps"
raw_skymap_path = raw_skymaps_dir / "skyMap_lsst_cells_v1_skymaps.pickle"

In [6]:
lsst_skymap = load_pickle_skymap(raw_skymap_path)
lsst_skymap

### Using the ConvertedSkymap writer

In [7]:
from skymap_convert.skymap_writers import ConvertedSkymapWriter

# Create a writer for the LSST skymap
writer = ConvertedSkymapWriter()

# Designate the output path for the converted skymap
converted_skymap_path = package_root / "converted_skymaps" / "lsst_skymap"

# Write the skymap to a file - warning: this may take a few minutes!
writer.write(lsst_skymap, output_path=converted_skymap_path)

Wrote Converted Skymap to: /sdf/home/o/olynn/skymap-convert/converted_skymaps/lsst_skymap


Let's take a quick look at the files we created.

In [8]:
!ls -lh {converted_skymap_path}

total 29M
-rw-r--r-- 1 olynn gu  123 Jul 14 07:29 metadata.yaml
-rw-r--r-- 1 olynn gu  28M Jul 14 07:29 patches.npy.gz
-rw-r--r-- 1 olynn gu 1.2M Jul 14 07:29 tracts.npy


Note that our `patches.npy` has been gzipped--without this compression, the file would be about 1.2 MB, just a little too big to be hosted on GitHub. 

Don't worry; the skymap writer and reader will handle gzipping and un-gzipping under the hood.

## About  

**Author:** Olivia Lynn

**Last updated on:** July 14, 2025